In [ ]:
# Install required dependencies
import subprocess
import sys

def install_requirements():
    """Install all required packages for CausalXray framework."""
    packages = [
        'torch>=1.12.0',
        'torchvision>=0.13.0', 
        'numpy>=1.21.0',
        'scipy>=1.7.0',
        'scikit-learn>=1.1.0',
        'pandas>=1.4.0',
        'matplotlib>=3.5.0',
        'seaborn>=0.11.0',
        'pyyaml>=6.0',
        'tqdm>=4.64.0',
        'pydicom>=2.3.0',
        'opencv-python>=4.6.0',
        'pillow>=9.0.0',
        'plotly>=5.10.0',
        'captum>=0.5.0',
        'lime>=0.2.0.1',
        'shap>=0.41.0',
        'omegaconf>=2.2.0',
        'hydra-core>=1.2.0',
        'wandb>=0.13.0',
        'tensorboard>=2.9.0',
        'albumentations>=1.2.0',
        'imgaug>=0.4.0',
        'statsmodels>=0.13.0',
        'torchmetrics>=0.7.0',
        'pytorch-lightning>=1.5.0'
    ]
    
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✓ Installed {package}")
        except subprocess.CalledProcessError:
            print(f"✗ Failed to install {package}")

# Uncomment the following line to install dependencies
# install_requirements()

print("Setup completed! All required packages should now be installed.")


In [ ]:
# Import all necessary libraries
import os
import sys
import json
import yaml
import time
import warnings
import copy
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union, Callable, Any
from collections import defaultdict
import logging
from datetime import datetime

# Scientific computing
import numpy as np
import pandas as pd
from scipy import ndimage
import statsmodels.api as sm

# PyTorch and deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard.writer import SummaryWriter
import pytorch_lightning as pl
from torchmetrics import functional as tmF

# Image processing and visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from PIL import Image, ImageFile
import cv2
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Attribution and interpretability
from captum.attr import IntegratedGradients, LayerGradCam, LayerConductance
import shap
import lime

# Progress bars and utilities
from tqdm.auto import tqdm
import wandb

# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True
warnings.filterwarnings('ignore', category=UserWarning)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
print("Environment setup complete!")


In [ ]:
# Data Transforms for Chest X-ray Images
class CausalTransforms:
    """Image transforms optimized for chest X-ray analysis with causal considerations."""
    
    def __init__(
        self,
        mode: str = "train",
        image_size: Tuple[int, int] = (224, 224),
        mean: List[float] = [0.485, 0.456, 0.406],
        std: List[float] = [0.229, 0.224, 0.225],
        augment_prob: float = 0.8
    ):
        """
        Initialize transforms for different modes.
        
        Args:
            mode: Transform mode ('train', 'val', 'test')
            image_size: Target image size
            mean: Normalization mean values
            std: Normalization std values
            augment_prob: Probability of applying augmentations
        """
        self.mode = mode
        self.image_size = image_size
        self.mean = mean
        self.std = std
        
        if mode == "train":
            self.transforms = A.Compose([
                A.Resize(height=image_size[0] + 32, width=image_size[1] + 32),
                A.RandomCrop(height=image_size[0], width=image_size[1]),
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    shift_limit=0.1,
                    scale_limit=0.1, 
                    rotate_limit=10,
                    border_mode=cv2.BORDER_CONSTANT,
                    value=0,
                    p=0.7
                ),
                A.OneOf([
                    A.OpticalDistortion(distort_limit=0.1, p=0.5),
                    A.GridDistortion(distort_limit=0.1, p=0.5),
                    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5)
                ], p=0.3),
                A.OneOf([
                    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.8),
                    A.RandomGamma(gamma_limit=(80, 120), p=0.8),
                    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.8)
                ], p=0.5),
                A.Normalize(mean=mean, std=std),
                ToTensorV2()
            ])
        else:  # val or test
            self.transforms = A.Compose([
                A.Resize(height=image_size[0], width=image_size[1]),
                A.Normalize(mean=mean, std=std),
                ToTensorV2()
            ])
    
    def __call__(self, image):
        """Apply transforms to image."""
        if isinstance(image, Image.Image):
            image = np.array(image)
        
        if len(image.shape) == 2:  # Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        elif len(image.shape) == 3 and image.shape[2] == 1:  # Single channel
            image = np.repeat(image, 3, axis=2)
            
        # Apply albumentations transforms
        transformed = self.transforms(image=image)
        return transformed['image']

print("Data transforms defined successfully!")


In [ ]:
# Dataset Classes for Chest X-ray Data
class ChestXrayDataset(Dataset):
    """Base class for chest X-ray datasets with causal confounder support."""
    
    def __init__(
        self,
        data_dir: str,
        split: str = "train",
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        include_confounders: bool = True,
        confounder_config: Optional[Dict[str, Any]] = None
    ):
        """Initialize chest X-ray dataset."""
        self.data_dir = data_dir
        self.split = split
        self.transform = transform
        self.target_transform = target_transform
        self.include_confounders = include_confounders
        self.confounder_config = confounder_config or {}
        
        # Will be set by subclasses
        self.images = []
        self.labels = []
        self.confounders = []
        self.metadata = []
        
        # Label mappings
        self.class_to_idx = {"normal": 0, "pneumonia": 1}
        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}
        
        # Load dataset
        self._load_dataset()
    
    def _load_dataset(self):
        """Load dataset - to be implemented by subclasses."""
        raise NotImplementedError("Subclasses must implement _load_dataset")
    
    def __len__(self) -> int:
        return len(self.images)
    
    def __getitem__(self, idx: int) -> Dict[str, Union[torch.Tensor, Dict]]:
        """Get dataset item with image, label, and optional confounders."""
        # Load image
        image_path = self.images[idx]
        image = self._load_image(image_path)
        
        # Get label
        label = self.labels[idx]
        if isinstance(label, str):
            label = self.class_to_idx.get(label.lower(), 0)
        
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        
        if self.target_transform:
            label = self.target_transform(label)
        
        # Prepare output
        item = {
            'image': image,
            'label': torch.tensor(label, dtype=torch.long),
            'index': idx
        }
        
        # Add confounders if available
        if self.include_confounders and idx < len(self.confounders):
            confounders = self.confounders[idx]
            processed_confounders = self._process_confounders(confounders)
            item['confounders'] = processed_confounders
        
        # Add metadata
        if idx < len(self.metadata):
            item['metadata'] = self.metadata[idx]
        
        return item
    
    def _load_image(self, image_path: str) -> Image.Image:
        """Load image from file path."""
        try:
            image = Image.open(image_path)
            if image.mode != 'RGB':
                image = image.convert('RGB')
        except Exception as e:
            warnings.warn(f"Error loading image {image_path}: {e}")
            # Return a blank image as fallback
            image = Image.new('RGB', (224, 224), color='black')
        return image
    
    def _process_confounders(self, confounders: Dict[str, Any]) -> Dict[str, torch.Tensor]:
        """Process confounder values into tensors."""
        processed = {}
        
        for name, value in confounders.items():
            if isinstance(value, (int, float)):
                processed[name] = torch.tensor(float(value), dtype=torch.float32)
            elif isinstance(value, str):
                # Handle categorical confounders
                if name in self.confounder_config:
                    categories = self.confounder_config[name].get('categories', [])
                    if value in categories:
                        idx = categories.index(value)
                        processed[name] = torch.tensor(idx, dtype=torch.long)
                    else:
                        processed[name] = torch.tensor(0, dtype=torch.long)
                else:
                    try:
                        processed[name] = torch.tensor(float(value), dtype=torch.float32)
                    except ValueError:
                        processed[name] = torch.tensor(0, dtype=torch.long)
            elif isinstance(value, (list, np.ndarray)):
                processed[name] = torch.tensor(value, dtype=torch.float32)
            else:
                processed[name] = torch.tensor(0, dtype=torch.float32)
        
        return processed
    
    def get_class_weights(self) -> torch.Tensor:
        """Compute class weights for imbalanced datasets."""
        label_counts = np.bincount(self.labels)
        total_samples = len(self.labels)
        weights = total_samples / (len(label_counts) * label_counts)
        return torch.tensor(weights, dtype=torch.float32)


class NIHChestXray14(ChestXrayDataset):
    """NIH ChestX-ray14 dataset implementation."""
    
    def _load_dataset(self):
        """Load NIH ChestX-ray14 dataset."""
        # Sample implementation - adapt based on actual data structure
        images_dir = os.path.join(self.data_dir, 'images')
        labels_file = os.path.join(self.data_dir, f'{self.split}_list.txt')
        
        if not os.path.exists(images_dir):
            print(f"Warning: Images directory not found at {images_dir}")
            print("Creating sample data for demonstration...")
            self._create_sample_data()
            return
        
        # Load labels if available
        if os.path.exists(labels_file):
            with open(labels_file, 'r') as f:
                lines = f.readlines()
            
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 2:
                    image_path = os.path.join(images_dir, parts[0])
                    label = int(parts[1])
                    
                    if os.path.exists(image_path):
                        self.images.append(image_path)
                        self.labels.append(label)
                        
                        # Sample confounders
                        confounders = {
                            'age': np.random.randint(18, 90),
                            'sex': np.random.choice(['M', 'F']),
                            'view_position': np.random.choice(['PA', 'AP', 'L'])
                        }
                        self.confounders.append(confounders)
        else:
            self._create_sample_data()
    
    def _create_sample_data(self):
        """Create sample data for demonstration."""
        print("Creating sample dataset for demonstration purposes...")
        
        # Create sample images and labels
        n_samples = 100 if self.split == 'train' else 30
        
        for i in range(n_samples):
            # Create dummy image path
            image_path = f"sample_image_{i}.jpg"
            label = np.random.choice([0, 1])  # Binary classification
            
            self.images.append(image_path)
            self.labels.append(label)
            
            # Sample confounders
            confounders = {
                'age': np.random.randint(18, 90),
                'sex': np.random.choice(['M', 'F']),
                'view_position': np.random.choice(['PA', 'AP', 'L'])
            }
            self.confounders.append(confounders)
    
    def _load_image(self, image_path: str) -> Image.Image:
        """Load image, creating sample if not found."""
        if not os.path.exists(image_path):
            # Create a sample chest X-ray-like image
            img_array = np.random.randint(0, 256, (256, 256, 3), dtype=np.uint8)
            # Add some structure to make it look more like an X-ray
            img_array[:, :, 0] = img_array[:, :, 1] = img_array[:, :, 2] = \
                np.random.randint(50, 200, (256, 256))
            return Image.fromarray(img_array)
        else:
            return super()._load_image(image_path)


class RSNAPneumonia(ChestXrayDataset):
    """RSNA Pneumonia dataset implementation."""
    
    def _load_dataset(self):
        """Load RSNA dataset."""
        # Similar implementation to NIH but adapted for RSNA structure
        self._create_sample_data()
    
    def _create_sample_data(self):
        """Create sample RSNA data."""
        print("Creating sample RSNA dataset...")
        n_samples = 80 if self.split == 'train' else 25
        
        for i in range(n_samples):
            image_path = f"rsna_sample_image_{i}.jpg"
            label = np.random.choice([0, 1])
            
            self.images.append(image_path)
            self.labels.append(label)
            
            confounders = {
                'age': np.random.randint(0, 100),  # Includes pediatric
                'sex': np.random.choice(['M', 'F']),
                'scanner_type': np.random.choice(['A', 'B', 'C'])
            }
            self.confounders.append(confounders)


class PediatricDataset(ChestXrayDataset):
    """Pediatric chest X-ray dataset implementation."""
    
    def _load_dataset(self):
        """Load pediatric dataset."""
        self._create_sample_data()
    
    def _create_sample_data(self):
        """Create sample pediatric data."""
        print("Creating sample pediatric dataset...")
        n_samples = 60 if self.split == 'train' else 20
        
        for i in range(n_samples):
            image_path = f"pediatric_sample_image_{i}.jpg"
            label = np.random.choice([0, 1])
            
            self.images.append(image_path)
            self.labels.append(label)
            
            confounders = {
                'age': np.random.randint(0, 18),  # Pediatric only
                'sex': np.random.choice(['M', 'F']),
                'weight': np.random.uniform(2.5, 70.0)  # kg
            }
            self.confounders.append(confounders)


def create_dataloader(dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True):
    """Create DataLoader for the given dataset."""
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory
    )

print("Dataset classes defined successfully!")


In [ ]:
# CausalXray Model Architecture
class CausalBackbone(nn.Module):
    """CNN backbone with causal integration capabilities."""
    
    def __init__(
        self,
        architecture: str = "densenet121",
        pretrained: bool = True,
        num_classes: int = 2,
        feature_dims: List[int] = [1024, 512, 256],
        dropout_rate: float = 0.3
    ):
        """
        Initialize the causal backbone network.
        
        Args:
            architecture: CNN architecture ("densenet121" or "resnet50")
            pretrained: Whether to use ImageNet pretrained weights
            num_classes: Number of output classes
            feature_dims: Dimensions for intermediate feature layers
            dropout_rate: Dropout probability for regularization
        """
        super(CausalBackbone, self).__init__()
        
        self.architecture = architecture
        self.num_classes = num_classes
        self.feature_dims = feature_dims
        
        # Initialize base CNN architecture
        if architecture == "densenet121":
            self.backbone = models.densenet121(pretrained=pretrained)
            self.feature_size = self.backbone.classifier.in_features
            # Replace classifier with identity
            self.backbone.classifier = nn.Linear(self.feature_size, self.feature_size)
            
        elif architecture == "resnet50":
            self.backbone = models.resnet50(pretrained=pretrained)
            self.feature_size = self.backbone.fc.in_features
            # Replace fc with identity
            self.backbone.fc = nn.Linear(self.feature_size, self.feature_size)
            
        else:
            raise ValueError(f"Unsupported architecture: {architecture}")
        
        # Causal-aware feature processing layers
        self.causal_features = self._build_causal_layers()
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(feature_dims[-1], feature_dims[-1] // 2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(feature_dims[-1] // 2, num_classes)
        )
        
        # Initialize weights for new layers
        self._initialize_weights()
    
    def _build_causal_layers(self) -> nn.ModuleList:
        """Build causal-aware feature processing layers."""
        layers = nn.ModuleList()
        
        input_dim = self.feature_size
        for output_dim in self.feature_dims:
            layers.append(nn.Sequential(
                nn.Linear(input_dim, output_dim),
                nn.BatchNorm1d(output_dim),
                nn.ReLU(inplace=True),
                nn.Dropout(0.2)
            ))
            input_dim = output_dim
        
        return layers
    
    def _initialize_weights(self):
        """Initialize weights for newly added layers."""
        for module in [self.causal_features, self.classifier]:
            for m in module.modules():
                if isinstance(m, nn.Linear):
                    nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
                elif isinstance(m, nn.BatchNorm1d):
                    nn.init.constant_(m.weight, 1)
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x: torch.Tensor) -> Dict[str, Any]:
        """
        Forward pass through the causal backbone.
        
        Args:
            x: Input tensor of shape (batch_size, channels, height, width)
            
        Returns:
            Dictionary containing features, logits, and probabilities
        """
        # Extract raw backbone features
        raw_features = self.backbone(x)
        
        # Process through causal layers
        causal_features = []
        current_features = raw_features
        
        for layer in self.causal_features:
            current_features = layer(current_features)
            causal_features.append(current_features)
        
        # Generate classification predictions
        logits = self.classifier(causal_features[-1])
        probabilities = torch.softmax(logits, dim=1)
        
        return {
            'features': raw_features,
            'causal_features': causal_features,
            'logits': logits,
            'probabilities': probabilities
        }
    
    def get_feature_maps(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """Extract feature maps for attribution analysis."""
        if self.architecture == "densenet121":
            return self._extract_densenet_features(x)
        elif self.architecture == "resnet50":
            return self._extract_resnet_features(x)
        else:
            return {}
    
    def _extract_densenet_features(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """Extract DenseNet feature maps."""
        feature_maps = {}
        features = self.backbone.features
        
        for i, module in enumerate(features):
            x = module(x)
            if isinstance(module, nn.ReLU):
                feature_maps[f'layer_{i}'] = x
        
        return feature_maps
    
    def _extract_resnet_features(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """Extract ResNet feature maps."""
        feature_maps = {}
        
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        feature_maps['layer1'] = x
        
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        feature_maps['layer2'] = x
        
        x = self.backbone.layer2(x)
        feature_maps['layer3'] = x
        
        x = self.backbone.layer3(x)
        feature_maps['layer4'] = x
        
        x = self.backbone.layer4(x)
        feature_maps['layer5'] = x
        
        return feature_maps


# Causal Heads for confounder prediction
class CausalHeads(nn.Module):
    """Causal heads for disentanglement of confounders."""
    
    def __init__(
        self,
        feature_dim: int,
        confounder_dims: Dict[str, int],
        hidden_dims: List[int] = [256, 128]
    ):
        """
        Initialize causal heads.
        
        Args:
            feature_dim: Input feature dimension
            confounder_dims: Dictionary mapping confounder names to output dimensions
            hidden_dims: Hidden layer dimensions
        """
        super(CausalHeads, self).__init__()
        
        self.confounder_dims = confounder_dims
        self.heads = nn.ModuleDict()
        
        for name, output_dim in confounder_dims.items():
            layers = []
            input_dim = feature_dim
            
            # Hidden layers
            for hidden_dim in hidden_dims:
                layers.extend([
                    nn.Linear(input_dim, hidden_dim),
                    nn.ReLU(inplace=True),
                    nn.Dropout(0.2)
                ])
                input_dim = hidden_dim
            
            # Output layer
            layers.append(nn.Linear(input_dim, output_dim))
            
            self.heads[name] = nn.Sequential(*layers)
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize weights."""
        for head in self.heads.values():
            for m in head.modules():
                if isinstance(m, nn.Linear):
                    nn.init.kaiming_normal_(m.weight)
                    if m.bias is not None:
                        nn.init.constant_(m.bias, 0)
    
    def forward(self, features: torch.Tensor) -> Dict[str, torch.Tensor]:
        """Forward pass through causal heads."""
        outputs = {}
        
        for name, head in self.heads.items():
            outputs[name] = head(features)
        
        return outputs


# Complete CausalXray Model
class CausalXrayModel(nn.Module):
    """Complete CausalXray model with backbone and causal heads."""
    
    def __init__(
        self,
        backbone_config: Dict[str, Any],
        causal_config: Dict[str, Any]
    ):
        """Initialize complete CausalXray model."""
        super(CausalXrayModel, self).__init__()
        
        # Backbone network
        self.backbone = CausalBackbone(**backbone_config)
        
        # Causal heads for confounder prediction
        if causal_config.get('confounders'):
            self.causal_heads = CausalHeads(
                feature_dim=backbone_config['feature_dims'][-1],
                confounder_dims=causal_config['confounders'],
                hidden_dims=causal_config.get('hidden_dims', [256, 128])
            )
        else:
            self.causal_heads = None
        
        self.training_phase = 'full'  # 'backbone', 'causal', 'full'
    
    def set_training_phase(self, phase: str):
        """Set training phase for progressive training."""
        self.training_phase = phase
        
        if phase == 'backbone':
            # Only train backbone
            for param in self.backbone.parameters():
                param.requires_grad = True
            if self.causal_heads:
                for param in self.causal_heads.parameters():
                    param.requires_grad = False
                    
        elif phase == 'causal':
            # Only train causal heads
            for param in self.backbone.parameters():
                param.requires_grad = False
            if self.causal_heads:
                for param in self.causal_heads.parameters():
                    param.requires_grad = True
                    
        else:  # 'full'
            # Train everything
            for param in self.parameters():
                param.requires_grad = True
    
    def forward(self, x: torch.Tensor) -> Dict[str, Any]:
        """Forward pass through complete model."""
        # Backbone forward pass
        backbone_output = self.backbone(x)
        
        outputs = {
            'probabilities': backbone_output['probabilities'],
            'logits': backbone_output['logits'],
            'features': backbone_output['features'],
            'causal_features': backbone_output['causal_features']
        }
        
        # Causal heads prediction
        if self.causal_heads and self.training_phase in ['causal', 'full']:
            confounder_preds = self.causal_heads(backbone_output['causal_features'][-1])
            outputs['confounders'] = confounder_preds
        
        return outputs

print("Model architecture defined successfully!")


In [ ]:
# Loss Functions for CausalXray Training
class CausalLoss(nn.Module):
    """Multi-objective loss function for causal training."""
    
    def __init__(self, config: Dict[str, Any]):
        """Initialize causal loss."""
        super(CausalLoss, self).__init__()
        
        self.config = config
        self.weights = config.get('weights', {
            'classification': 1.0,
            'disentanglement': 0.3,
            'domain': 0.1,
            'attribution': 0.2
        })
        
        # Classification loss
        self.classification_loss = nn.CrossEntropyLoss()
        
        # Focal loss for imbalanced datasets
        focal_alpha = config.get('focal_alpha', 1.0)
        focal_gamma = config.get('focal_gamma', 2.0)
        self.focal_loss = FocalLoss(alpha=focal_alpha, gamma=focal_gamma)
        
        self.use_focal = config.get('use_focal', False)
    
    def forward(
        self,
        predictions: Dict[str, torch.Tensor],
        targets: Dict[str, torch.Tensor]
    ) -> Dict[str, torch.Tensor]:
        """Compute multi-objective loss."""
        losses = {}
        total_loss = 0.0
        
        # Classification loss
        if self.use_focal:
            class_loss = self.focal_loss(predictions['logits'], targets['labels'])
        else:
            class_loss = self.classification_loss(predictions['logits'], targets['labels'])
        
        losses['classification'] = class_loss
        total_loss += self.weights['classification'] * class_loss
        
        # Confounder disentanglement loss
        if 'confounders' in predictions and 'confounders' in targets:
            disentangle_loss = self._compute_disentanglement_loss(
                predictions['confounders'], targets['confounders']
            )
            losses['disentanglement'] = disentangle_loss
            total_loss += self.weights['disentanglement'] * disentangle_loss
        
        # Domain adaptation loss (if applicable)
        if 'domain_logits' in predictions and 'domains' in targets:
            domain_loss = self.classification_loss(predictions['domain_logits'], targets['domains'])
            losses['domain'] = domain_loss
            total_loss += self.weights['domain'] * domain_loss
        
        losses['total'] = total_loss
        return losses
    
    def _compute_disentanglement_loss(
        self,
        pred_confounders: Dict[str, torch.Tensor],
        true_confounders: Dict[str, torch.Tensor]
    ) -> torch.Tensor:
        """Compute disentanglement loss for confounders."""
        total_loss = 0.0
        count = 0
        
        for name in pred_confounders:
            if name in true_confounders:
                pred = pred_confounders[name]
                true = true_confounders[name]
                
                if len(true.shape) == 1 and pred.shape[-1] > 1:
                    # Categorical confounder
                    loss = nn.CrossEntropyLoss()(pred, true.long())
                else:
                    # Continuous confounder
                    loss = nn.MSELoss()(pred.squeeze(), true.float())
                
                total_loss += loss
                count += 1
        
        return total_loss / count if count > 0 else torch.tensor(0.0, device=total_loss.device if count > 0 else 'cpu')


class FocalLoss(nn.Module):
    """Focal Loss for addressing class imbalance."""
    
    def __init__(self, alpha: float = 1.0, gamma: float = 2.0, reduction: str = 'mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        """Compute focal loss."""
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


# Metrics for Evaluation
class CausalMetrics:
    """Comprehensive metrics for causal model evaluation."""
    
    def __init__(self):
        """Initialize metrics calculator."""
        pass
    
    def compute_classification_metrics(
        self,
        y_true: np.ndarray,
        y_pred: np.ndarray,
        y_prob: Optional[np.ndarray] = None
    ) -> Dict[str, float]:
        """Compute classification metrics."""
        metrics_dict = {}
        
        # Basic metrics
        metrics_dict['accuracy'] = metrics.accuracy_score(y_true, y_pred)
        metrics_dict['precision'] = metrics.precision_score(y_true, y_pred, average='weighted', zero_division=0)
        metrics_dict['recall'] = metrics.recall_score(y_true, y_pred, average='weighted', zero_division=0)
        metrics_dict['f1'] = metrics.f1_score(y_true, y_pred, average='weighted', zero_division=0)
        
        # AUC metrics if probabilities provided
        if y_prob is not None:
            if y_prob.shape[1] == 2:  # Binary classification
                metrics_dict['auc'] = metrics.roc_auc_score(y_true, y_prob[:, 1])
                metrics_dict['ap'] = metrics.average_precision_score(y_true, y_prob[:, 1])
            else:  # Multi-class
                try:
                    metrics_dict['auc'] = metrics.roc_auc_score(y_true, y_prob, multi_class='ovr')
                except ValueError:
                    metrics_dict['auc'] = 0.0
        
        # Confusion matrix derived metrics
        cm = metrics.confusion_matrix(y_true, y_pred)
        if cm.shape == (2, 2):  # Binary case
            tn, fp, fn, tp = cm.ravel()
            metrics_dict['sensitivity'] = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            metrics_dict['specificity'] = tn / (tn + fp) if (tn + fp) > 0 else 0.0
            metrics_dict['npv'] = tn / (tn + fn) if (tn + fn) > 0 else 0.0
            metrics_dict['ppv'] = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        
        return metrics_dict
    
    def compute_epoch_metrics(
        self,
        predictions: np.ndarray,
        labels: np.ndarray,
        return_detailed: bool = True
    ) -> Dict[str, float]:
        """Compute comprehensive metrics for an epoch."""
        # Get predicted classes
        if predictions.ndim == 2 and predictions.shape[1] > 1:
            y_pred = np.argmax(predictions, axis=1)
            y_prob = predictions
        else:
            y_pred = predictions.astype(int)
            y_prob = None
        
        # Compute metrics
        epoch_metrics = self.compute_classification_metrics(labels, y_pred, y_prob)
        
        if return_detailed:
            # Additional detailed metrics
            epoch_metrics['balanced_accuracy'] = metrics.balanced_accuracy_score(labels, y_pred)
            
            # Per-class metrics
            if len(np.unique(labels)) == 2:  # Binary
                class_report = metrics.classification_report(
                    labels, y_pred, target_names=['Normal', 'Pneumonia'], output_dict=True, zero_division=0
                )
                for class_name in ['Normal', 'Pneumonia']:
                    if class_name in class_report:
                        epoch_metrics[f'{class_name.lower()}_precision'] = class_report[class_name]['precision']
                        epoch_metrics[f'{class_name.lower()}_recall'] = class_report[class_name]['recall']
                        epoch_metrics[f'{class_name.lower()}_f1'] = class_report[class_name]['f1-score']
        
        return epoch_metrics
    
    def compute_attribution_metrics(
        self,
        attributions: Dict[str, np.ndarray],
        ground_truth_masks: Optional[np.ndarray] = None
    ) -> Dict[str, float]:
        """Compute attribution quality metrics."""
        attr_metrics = {}
        
        if ground_truth_masks is not None:
            for method, attr in attributions.items():
                # Localization metrics
                attr_flat = attr.flatten()
                gt_flat = ground_truth_masks.flatten()
                
                # Spearman correlation
                try:
                    from scipy.stats import spearmanr
                    corr, p_value = spearmanr(attr_flat, gt_flat)
                    attr_metrics[f'{method}_correlation'] = corr if not np.isnan(corr) else 0.0
                except:
                    attr_metrics[f'{method}_correlation'] = 0.0
                
                # Intersection over Union (IoU) for top-k attributions
                k = int(0.1 * len(attr_flat))  # Top 10%
                top_k_indices = np.argpartition(attr_flat, -k)[-k:]
                gt_positive_indices = np.where(gt_flat > 0.5)[0]
                
                intersection = len(np.intersect1d(top_k_indices, gt_positive_indices))
                union = len(np.union1d(top_k_indices, gt_positive_indices))
                
                attr_metrics[f'{method}_iou'] = intersection / union if union > 0 else 0.0
        
        return attr_metrics

print("Loss functions and metrics defined successfully!")


In [ ]:
# CausalXray Trainer
class CausalTrainer:
    """Main trainer class for CausalXray model with progressive training."""
    
    def __init__(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        config: Dict[str, Any],
        device: str = "cuda",
        logger: Optional[logging.Logger] = None
    ):
        """Initialize CausalXray trainer."""
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.config = config
        self.device = device
        self.logger = logger or self._setup_logger()
        
        # Training components
        self.criterion = CausalLoss(config.get('loss', {}))
        self.optimizer = self._setup_optimizer()
        self.scheduler = self._setup_scheduler()
        self.metrics = CausalMetrics()
        
        # Training state
        self.current_epoch = 0
        self.best_metric = 0.0
        self.training_history = defaultdict(list)
        
        # Progressive training configuration
        self.progressive_config = config.get('progressive_training', {})
        self.phase_epochs = self.progressive_config.get('phase_epochs', [50, 50, 50])
        
        # Create output directories
        self.output_dir = Path(config.get('output_dir', './outputs'))
        self.checkpoint_dir = self.output_dir / 'checkpoints'
        self.log_dir = self.output_dir / 'logs'
        
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)
        self.log_dir.mkdir(parents=True, exist_ok=True)
    
    def _setup_logger(self) -> logging.Logger:
        """Setup logger."""
        logger = logging.getLogger('CausalTrainer')
        logger.setLevel(logging.INFO)
        
        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            logger.addHandler(handler)
        
        return logger
    
    def _setup_optimizer(self) -> optim.Optimizer:
        """Setup optimizer."""
        optimizer_config = self.config.get('optimizer', {})
        optimizer_type = optimizer_config.get('type', 'adam').lower()
        
        if optimizer_type == 'adam':
            return optim.Adam(
                self.model.parameters(),
                lr=optimizer_config.get('lr', 1e-3),
                weight_decay=optimizer_config.get('weight_decay', 1e-4)
            )
        elif optimizer_type == 'adamw':
            return optim.AdamW(
                self.model.parameters(),
                lr=optimizer_config.get('lr', 1e-3),
                weight_decay=optimizer_config.get('weight_decay', 1e-4)
            )
        elif optimizer_type == 'sgd':
            return optim.SGD(
                self.model.parameters(),
                lr=optimizer_config.get('lr', 1e-2),
                momentum=optimizer_config.get('momentum', 0.9),
                weight_decay=optimizer_config.get('weight_decay', 1e-4)
            )
        else:
            raise ValueError(f"Unsupported optimizer: {optimizer_type}")
    
    def _setup_scheduler(self) -> Optional[Any]:
        """Setup learning rate scheduler."""
        scheduler_config = self.config.get('scheduler', {})
        if not scheduler_config.get('enabled', False):
            return None
        
        scheduler_type = scheduler_config.get('type', 'cosine')
        
        if scheduler_type == 'cosine':
            return optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer,
                T_max=scheduler_config.get('T_max', 100),
                eta_min=scheduler_config.get('eta_min', 1e-6)
            )
        elif scheduler_type == 'step':
            return optim.lr_scheduler.StepLR(
                self.optimizer,
                step_size=scheduler_config.get('step_size', 30),
                gamma=scheduler_config.get('gamma', 0.1)
            )
        elif scheduler_type == 'plateau':
            return optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer,
                mode='max',
                factor=scheduler_config.get('factor', 0.5),
                patience=scheduler_config.get('patience', 10)
            )
        else:
            return None
    
    def _get_training_phase(self, epoch: int) -> str:
        """Determine current training phase based on epoch."""
        if not self.progressive_config.get('enabled', True):
            return 'full'
        
        cumulative_epochs = np.cumsum(self.phase_epochs)
        if epoch < cumulative_epochs[0]:
            return 'backbone'
        elif epoch < cumulative_epochs[1]:
            return 'causal'
        else:
            return 'full'\n    \n    def train(self, num_epochs: int, resume_from: Optional[str] = None) -> Dict[str, List]:\n        \"\"\"Main training loop with progressive training.\"\"\"\n        if resume_from:\n            self._load_checkpoint(resume_from)\n        \n        self.logger.info(f\"Starting training for {num_epochs} epochs\")\n        self.logger.info(f\"Progressive training phases: {self.phase_epochs}\")\n        \n        start_time = time.time()\n        \n        for epoch in range(self.current_epoch, num_epochs):\n            self.current_epoch = epoch\n            \n            # Determine training phase\n            phase = self._get_training_phase(epoch)\n            if hasattr(self.model, 'set_training_phase'):\n                self.model.set_training_phase(phase)\n                if epoch == 0 or self._get_training_phase(epoch-1) != phase:\n                    self.logger.info(f\"Switched to training phase: {phase}\")\n            \n            # Training step\n            train_metrics = self._train_epoch()\n            \n            # Validation step\n            val_metrics = self._validate_epoch()\n            \n            # Update learning rate\n            if self.scheduler:\n                if isinstance(self.scheduler, optim.lr_scheduler.ReduceLROnPlateau):\n                    self.scheduler.step(val_metrics.get('auc', val_metrics.get('accuracy', 0)))\n                else:\n                    self.scheduler.step()\n            \n            # Log metrics\n            self._log_metrics(train_metrics, val_metrics, epoch)\n            \n            # Save checkpoint\n            if self._should_save_checkpoint(val_metrics):\n                self._save_checkpoint(epoch, val_metrics)\n            \n            # Early stopping check\n            if self._should_early_stop(val_metrics):\n                self.logger.info(f\"Early stopping triggered at epoch {epoch}\")\n                break\n        \n        training_time = time.time() - start_time\n        self.logger.info(f\"Training completed in {training_time:.2f} seconds\")\n        \n        return dict(self.training_history)\n    \n    def _train_epoch(self) -> Dict[str, float]:\n        \"\"\"Train for one epoch.\"\"\"\n        self.model.train()\n        \n        running_losses = defaultdict(float)\n        all_predictions = []\n        all_labels = []\n        \n        progress_bar = tqdm(self.train_loader, desc=f\"Epoch {self.current_epoch} [Train]\")\n        \n        for batch_idx, batch in enumerate(progress_bar):\n            # Move batch to device\n            images = batch['image'].to(self.device)\n            labels = batch['label'].to(self.device)\n            \n            # Prepare targets\n            targets = {'labels': labels}\n            if 'confounders' in batch:\n                confounders = {}\n                for name, values in batch['confounders'].items():\n                    confounders[name] = values.to(self.device)\n                targets['confounders'] = confounders\n            \n            # Forward pass\n            self.optimizer.zero_grad()\n            predictions = self.model(images)\n            \n            # Compute losses\n            losses = self.criterion(predictions, targets)\n            \n            # Backward pass\n            losses['total'].backward()\n            \n            # Gradient clipping\n            if self.config.get('grad_clip', 0) > 0:\n                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config['grad_clip'])\n            \n            self.optimizer.step()\n            \n            # Accumulate losses\n            for loss_name, loss_value in losses.items():\n                running_losses[loss_name] += loss_value.item()\n            \n            # Store predictions for metrics\n            all_predictions.append(predictions['probabilities'].detach().cpu().numpy())\n            all_labels.append(labels.cpu().numpy())\n            \n            # Update progress bar\n            progress_bar.set_postfix({\n                'loss': f\"{running_losses['total']/(batch_idx+1):.4f}\",\n                'lr': f\"{self.optimizer.param_groups[0]['lr']:.2e}\"\n            })\n        \n        # Calculate epoch metrics\n        epoch_losses = {k: v / len(self.train_loader) for k, v in running_losses.items()}\n        \n        # Classification metrics\n        all_predictions = np.vstack(all_predictions)\n        all_labels = np.concatenate(all_labels)\n        \n        classification_metrics = self.metrics.compute_epoch_metrics(all_predictions, all_labels)\n        \n        # Combine all metrics\n        train_metrics = {**epoch_losses, **classification_metrics}\n        \n        return train_metrics\n    \n    def _validate_epoch(self) -> Dict[str, float]:\n        \"\"\"Validate for one epoch.\"\"\"\n        self.model.eval()\n        \n        running_losses = defaultdict(float)\n        all_predictions = []\n        all_labels = []\n        \n        with torch.no_grad():\n            for batch in tqdm(self.val_loader, desc=f\"Epoch {self.current_epoch} [Val]\"):\n                # Move batch to device\n                images = batch['image'].to(self.device)\n                labels = batch['label'].to(self.device)\n                \n                # Prepare targets\n                targets = {'labels': labels}\n                if 'confounders' in batch:\n                    confounders = {}\n                    for name, values in batch['confounders'].items():\n                        confounders[name] = values.to(self.device)\n                    targets['confounders'] = confounders\n                \n                # Forward pass\n                predictions = self.model(images)\n                \n                # Compute losses\n                losses = self.criterion(predictions, targets)\n                \n                # Accumulate losses\n                for loss_name, loss_value in losses.items():\n                    running_losses[loss_name] += loss_value.item()\n                \n                # Store predictions for metrics\n                all_predictions.append(predictions['probabilities'].cpu().numpy())\n                all_labels.append(labels.cpu().numpy())\n        \n        # Calculate epoch metrics\n        epoch_losses = {f'val_{k}': v / len(self.val_loader) for k, v in running_losses.items()}\n        \n        # Classification metrics\n        all_predictions = np.vstack(all_predictions)\n        all_labels = np.concatenate(all_labels)\n        \n        classification_metrics = self.metrics.compute_epoch_metrics(all_predictions, all_labels)\n        val_classification_metrics = {f'val_{k}': v for k, v in classification_metrics.items()}\n        \n        # Combine all metrics\n        val_metrics = {**epoch_losses, **val_classification_metrics}\n        \n        return val_metrics\n    \n    def _log_metrics(self, train_metrics: Dict[str, float], val_metrics: Dict[str, float], epoch: int):\n        \"\"\"Log training metrics.\"\"\"\n        # Store in history\n        for metric_name, value in train_metrics.items():\n            self.training_history[f'train_{metric_name}'].append(value)\n        \n        for metric_name, value in val_metrics.items():\n            self.training_history[metric_name].append(value)\n        \n        # Log key metrics\n        self.logger.info(f\"Epoch {epoch}:\")\n        self.logger.info(f\"  Train - Loss: {train_metrics['total']:.4f}, Acc: {train_metrics['accuracy']:.4f}\")\n        self.logger.info(f\"  Val   - Loss: {val_metrics['val_total']:.4f}, Acc: {val_metrics['val_accuracy']:.4f}\")\n        \n        if 'auc' in train_metrics and 'val_auc' in val_metrics:\n            self.logger.info(f\"  Train AUC: {train_metrics['auc']:.4f}, Val AUC: {val_metrics['val_auc']:.4f}\")\n    \n    def _should_save_checkpoint(self, val_metrics: Dict[str, float]) -> bool:\n        \"\"\"Determine if checkpoint should be saved.\"\"\"\n        current_metric = val_metrics.get('val_auc', val_metrics.get('val_accuracy', 0))\n        \n        if current_metric > self.best_metric:\n            self.best_metric = current_metric\n            return True\n        \n        return False\n    \n    def _should_early_stop(self, val_metrics: Dict[str, float]) -> bool:\n        \"\"\"Determine if training should stop early.\"\"\"\n        if not self.config.get('early_stopping', False):\n            return False\n        \n        patience = self.config.get('patience', 20)\n        min_delta = self.config.get('min_delta', 0.001)\n        \n        # Simple early stopping based on validation metric\n        current_metric = val_metrics.get('val_auc', val_metrics.get('val_accuracy', 0))\n        \n        # Check if we have enough history\n        history_key = 'val_auc' if 'val_auc' in val_metrics else 'val_accuracy'\n        if len(self.training_history[history_key]) < patience:\n            return False\n        \n        # Check if metric has improved in last 'patience' epochs\n        recent_metrics = self.training_history[history_key][-patience:]\n        best_recent = max(recent_metrics)\n        \n        return (current_metric - best_recent) < min_delta\n    \n    def _save_checkpoint(self, epoch: int, val_metrics: Dict[str, float]):\n        \"\"\"Save model checkpoint.\"\"\"\n        checkpoint = {\n            'epoch': epoch,\n            'model_state_dict': self.model.state_dict(),\n            'optimizer_state_dict': self.optimizer.state_dict(),\n            'best_metric': self.best_metric,\n            'val_metrics': val_metrics,\n            'training_history': dict(self.training_history),\n            'config': self.config\n        }\n        \n        if self.scheduler:\n            checkpoint['scheduler_state_dict'] = self.scheduler.state_dict()\n        \n        # Save best model\n        checkpoint_path = self.checkpoint_dir / 'best_model.pth'\n        torch.save(checkpoint, checkpoint_path)\n        \n        self.logger.info(f\"Saved checkpoint to {checkpoint_path}\")\n    \n    def _load_checkpoint(self, checkpoint_path: str):\n        \"\"\"Load model checkpoint.\"\"\"\n        checkpoint = torch.load(checkpoint_path, map_location=self.device)\n        \n        self.model.load_state_dict(checkpoint['model_state_dict'])\n        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])\n        \n        if self.scheduler and 'scheduler_state_dict' in checkpoint:\n            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])\n        \n        self.current_epoch = checkpoint['epoch'] + 1\n        self.best_metric = checkpoint['best_metric']\n        self.training_history = defaultdict(list, checkpoint.get('training_history', {}))\n        \n        self.logger.info(f\"Loaded checkpoint from {checkpoint_path} (epoch {checkpoint['epoch']})\")\n\nprint(\"Training framework defined successfully!\")"


In [ ]:
# Causal Attribution Methods
class CausalAttribution(nn.Module):
    """Causal attribution module implementing intervention-based explanations."""
    
    def __init__(
        self,
        model: nn.Module,
        feature_layers: List[str] = None,
        attribution_methods: List[str] = ['intervention', 'counterfactual', 'gradcam'],
        patch_size: int = 16,
        num_patches: Optional[int] = None
    ):
        """Initialize causal attribution module."""
        super(CausalAttribution, self).__init__()
        
        self.model = model
        self.feature_layers = feature_layers or []
        self.attribution_methods = attribution_methods
        self.patch_size = patch_size
        self.num_patches = num_patches
        
        # Initialize attribution methods
        self.attributors = {}
        if 'gradcam' in attribution_methods:
            try:
                gradcam_layer = self._get_gradcam_layer(model)
                self.attributors['gradcam'] = LayerGradCam(model, gradcam_layer)
            except:
                print("Warning: Could not initialize GradCAM")
        
        if 'integrated_gradients' in attribution_methods:
            self.attributors['integrated_gradients'] = IntegratedGradients(model)
    
    def forward(
        self,
        x: torch.Tensor,
        target_class: Optional[int] = None,
        return_intermediate: bool = False
    ) -> Dict[str, torch.Tensor]:
        """Generate causal attributions for input images."""
        batch_size, channels, height, width = x.shape
        device = x.device
        
        # Get model predictions
        with torch.no_grad():
            model_output = self.model(x)
            if target_class is None:
                target_class_tensor = torch.argmax(model_output['probabilities'], dim=1)
            else:
                if isinstance(target_class, int):
                    target_class_tensor = torch.full((batch_size,), target_class, dtype=torch.long, device=device)
                elif isinstance(target_class, torch.Tensor):
                    target_class_tensor = target_class.to(device)
                else:
                    raise ValueError("target_class must be int, None, or torch.Tensor")
        
        attributions = {}
        
        # Intervention-based attribution
        if 'intervention' in self.attribution_methods:
            intervention_attr = self._intervention_attribution(x, target_class_tensor)
            attributions['intervention'] = intervention_attr
        
        # Counterfactual attribution
        if 'counterfactual' in self.attribution_methods:
            counterfactual_attr = self._counterfactual_attribution(x, target_class_tensor)
            attributions['counterfactual'] = counterfactual_attr
        
        # Traditional attribution methods for comparison
        if 'gradcam' in self.attribution_methods and 'gradcam' in self.attributors:
            gradcam_attr = self._gradcam_attribution(x, target_class_tensor)
            attributions['gradcam'] = gradcam_attr
        
        if 'integrated_gradients' in self.attribution_methods and 'integrated_gradients' in self.attributors:
            ig_attr = self._integrated_gradients_attribution(x, target_class_tensor)
            attributions['integrated_gradients'] = ig_attr
        
        # Aggregate attribution scores
        if len(attributions) > 1:
            aggregated_attr = self._aggregate_attributions(attributions)
            attributions['aggregated'] = aggregated_attr
        
        return attributions
    
    def _intervention_attribution(
        self,
        x: torch.Tensor,
        target_class: torch.Tensor
    ) -> torch.Tensor:
        """Compute intervention-based attribution using do-calculus."""
        batch_size, channels, height, width = x.shape
        device = x.device
        
        # Create patch grid
        patches_h = height // self.patch_size
        patches_w = width // self.patch_size
        
        # Initialize attribution map
        attribution_map = torch.zeros(batch_size, height, width, device=device)
        
        # Get baseline prediction
        with torch.no_grad():
            baseline_output = self.model(x)
            baseline_probs = baseline_output['probabilities']
        
        # Iterate through patches
        for i in range(patches_h):
            for j in range(patches_w):
                # Define patch boundaries
                h_start = i * self.patch_size
                h_end = min((i + 1) * self.patch_size, height)
                w_start = j * self.patch_size
                w_end = min((j + 1) * self.patch_size, width)
                
                # Create intervention (set patch to mean value)
                x_intervened = x.clone()
                patch_mean = torch.mean(x[:, :, h_start:h_end, w_start:w_end], dim=(2, 3), keepdim=True)
                x_intervened[:, :, h_start:h_end, w_start:w_end] = patch_mean
                
                # Compute intervened prediction
                with torch.no_grad():
                    intervened_output = self.model(x_intervened)
                    intervened_probs = intervened_output['probabilities']
                
                # Compute causal effect
                for b in range(batch_size):
                    target_idx = target_class[b].item()
                    causal_effect = baseline_probs[b, target_idx] - intervened_probs[b, target_idx]
                    attribution_map[b, h_start:h_end, w_start:w_end] = causal_effect
        
        return attribution_map
    
    def _counterfactual_attribution(
        self,
        x: torch.Tensor,
        target_class: torch.Tensor
    ) -> torch.Tensor:
        """Compute counterfactual attribution using structural causal models."""
        batch_size, channels, height, width = x.shape
        device = x.device
        
        # Initialize attribution map
        attribution_map = torch.zeros(batch_size, height, width, device=device)
        
        # Get model's causal representation
        with torch.no_grad():
            model_output = self.model(x)
        
        # Generate counterfactual scenarios
        patches_h = height // self.patch_size
        patches_w = width // self.patch_size
        
        for i in range(patches_h):
            for j in range(patches_w):
                h_start = i * self.patch_size
                h_end = min((i + 1) * self.patch_size, height)
                w_start = j * self.patch_size
                w_end = min((j + 1) * self.patch_size, width)
                
                # Create counterfactual image (replace patch with normal tissue pattern)
                x_counterfactual = x.clone()
                normal_patch = self._generate_normal_patch(x[:, :, h_start:h_end, w_start:w_end])
                x_counterfactual[:, :, h_start:h_end, w_start:w_end] = normal_patch
                
                # Compute counterfactual prediction
                with torch.no_grad():
                    cf_output = self.model(x_counterfactual)
                    cf_probs = cf_output['probabilities']
                    original_probs = model_output['probabilities']
                
                # Compute counterfactual effect
                for b in range(batch_size):
                    target_idx = target_class[b].item()
                    cf_effect = original_probs[b, target_idx] - cf_probs[b, target_idx]
                    attribution_map[b, h_start:h_end, w_start:w_end] = cf_effect
        
        return attribution_map
    
    def _generate_normal_patch(self, patch: torch.Tensor) -> torch.Tensor:
        """Generate a 'normal' version of a patch for counterfactual analysis."""
        # Simple implementation: use patch mean and add controlled noise
        patch_mean = torch.mean(patch, dim=(2, 3), keepdim=True)
        noise = torch.randn_like(patch) * 0.1 * torch.std(patch, dim=(2, 3), keepdim=True)
        normal_patch = patch_mean + noise
        
        # Clamp to valid pixel range
        normal_patch = torch.clamp(normal_patch, 0, 1)
        return normal_patch
    
    def _gradcam_attribution(
        self,
        x: torch.Tensor,
        target_class: torch.Tensor
    ) -> torch.Tensor:
        """Compute GradCAM attribution for comparison."""
        if 'gradcam' not in self.attributors:
            return torch.zeros(x.shape[0], x.shape[2], x.shape[3], device=x.device)
        
        attributions = []
        for i, target in enumerate(target_class):
            try:
                attr = self.attributors['gradcam'].attribute(
                    x[i:i+1], 
                    target=target.item()
                )
                attributions.append(attr.squeeze())
            except Exception as e:
                print(f"GradCAM attribution failed: {e}")
                attributions.append(torch.zeros(x.shape[2], x.shape[3], device=x.device))
        
        return torch.stack(attributions)
    
    def _integrated_gradients_attribution(
        self,
        x: torch.Tensor,
        target_class: torch.Tensor
    ) -> torch.Tensor:
        """Compute Integrated Gradients attribution for comparison."""
        if 'integrated_gradients' not in self.attributors:
            return torch.zeros_like(x)
        
        # Create baseline (typically zeros or mean image)
        baseline = torch.zeros_like(x)
        
        attributions = []
        for i, target in enumerate(target_class):
            try:
                attr = self.attributors['integrated_gradients'].attribute(
                    x[i:i+1],
                    baseline[i:i+1],
                    target=target.item(),
                    n_steps=50
                )
                # Sum across channels for visualization
                attr_summed = torch.sum(attr.squeeze(0), dim=0)
                attributions.append(attr_summed)
            except Exception as e:
                print(f"Integrated Gradients attribution failed: {e}")
                attributions.append(torch.zeros(x.shape[2], x.shape[3], device=x.device))
        
        return torch.stack(attributions)
    
    def _aggregate_attributions(
        self,
        attributions: Dict[str, torch.Tensor]
    ) -> torch.Tensor:
        """Aggregate multiple attribution maps into a single consensus map."""
        if not attributions:
            return torch.zeros(1)
        
        # Normalize each attribution map
        normalized_attrs = {}
        for method, attr_map in attributions.items():
            if method != 'aggregated':  # Avoid recursion
                # Normalize to [0, 1] range
                attr_flat = attr_map.view(attr_map.size(0), -1)
                attr_min = torch.min(attr_flat, dim=1)[0].unsqueeze(-1).unsqueeze(-1)
                attr_max = torch.max(attr_flat, dim=1)[0].unsqueeze(-1).unsqueeze(-1)
                
                attr_range = attr_max - attr_min
                attr_range[attr_range == 0] = 1  # Avoid division by zero
                
                normalized_attr = (attr_map - attr_min) / attr_range
                normalized_attrs[method] = normalized_attr
        
        if not normalized_attrs:
            return torch.zeros(1)
        
        # Weighted average (prioritize causal methods)
        weights = {
            'intervention': 0.4,
            'counterfactual': 0.4,
            'gradcam': 0.1,
            'integrated_gradients': 0.1
        }
        
        aggregated = torch.zeros_like(list(normalized_attrs.values())[0])
        total_weight = 0.0
        
        for method, attr_map in normalized_attrs.items():
            weight = weights.get(method, 0.1)
            aggregated += weight * attr_map
            total_weight += weight
        
        aggregated = aggregated / total_weight if total_weight > 0 else aggregated
        
        return aggregated
    
    def _get_gradcam_layer(self, model: nn.Module) -> nn.Module:
        """Helper to get the last convolutional layer for GradCAM."""
        # Try to find the backbone
        backbone = getattr(model, 'backbone', model)
        
        # For CausalXrayModel, get the actual backbone
        if hasattr(backbone, 'backbone'):
            backbone = backbone.backbone
        
        # For DenseNet
        if hasattr(backbone, 'features') and hasattr(backbone.features, 'denseblock4'):
            return backbone.features.denseblock4
        
        # For ResNet
        if hasattr(backbone, 'layer4'):
            return backbone.layer4
        
        # Fallback: try to get the last convolutional layer
        layers = list(backbone.modules())
        for layer in reversed(layers):
            if isinstance(layer, nn.Conv2d):
                return layer
        
        raise AttributeError("Could not find a suitable layer for GradCAM.")


# Visualization Tools
class AttributionVisualizer:
    """Visualization tools for causal attributions."""
    
    def __init__(self):
        """Initialize visualizer."""
        self.colormaps = {
            'intervention': 'Reds',
            'counterfactual': 'Blues',
            'gradcam': 'jet',
            'integrated_gradients': 'viridis',
            'aggregated': 'RdYlBu_r'
        }
    
    def visualize_attribution_comparison(
        self,
        original_image: np.ndarray,
        attributions: Dict[str, np.ndarray],
        prediction: Dict[str, Any],
        save_path: Optional[str] = None,
        figsize: Tuple[int, int] = (15, 10)
    ) -> plt.Figure:
        """Create a comparison visualization of different attribution methods."""
        n_methods = len(attributions)
        cols = min(n_methods + 1, 4)  # +1 for original image
        rows = (n_methods + 1) // cols + ((n_methods + 1) % cols > 0)
        
        fig, axes = plt.subplots(rows, cols, figsize=figsize)
        if rows == 1:
            axes = axes.reshape(1, -1) if cols > 1 else [axes]
        elif cols == 1:
            axes = axes.reshape(-1, 1)
        
        # Plot original image
        ax = axes[0, 0] if rows > 1 else axes[0]
        ax.imshow(original_image, cmap='gray')
        ax.set_title('Original Image')
        ax.axis('off')
        
        # Add prediction information
        pred_class = prediction.get('predicted_class', 'Unknown')
        confidence = prediction.get('confidence', 0.0)
        ax.text(0.02, 0.98, f'Prediction: {pred_class}\\nConfidence: {confidence:.3f}', 
                transform=ax.transAxes, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        # Plot attribution maps
        plot_idx = 1
        for method, attribution in attributions.items():
            row = plot_idx // cols
            col = plot_idx % cols
            
            if row < rows and col < cols:
                ax = axes[row, col] if rows > 1 else axes[col] if cols > 1 else axes
                
                # Normalize attribution for visualization
                attr_norm = self._normalize_attribution(attribution)
                
                # Create overlay
                overlay = self._create_attribution_overlay(original_image, attr_norm, method)
                
                ax.imshow(overlay)
                ax.set_title(f'{method.replace("_", " ").title()} Attribution')
                ax.axis('off')
            
            plot_idx += 1
        
        # Hide unused subplots
        for idx in range(plot_idx, rows * cols):
            row = idx // cols
            col = idx % cols
            if row < rows and col < cols:
                ax = axes[row, col] if rows > 1 else axes[col] if cols > 1 else axes
                ax.axis('off')
        
        plt.tight_layout()
        
        if save_path:
            fig.savefig(save_path, dpi=300, bbox_inches='tight')
        
        return fig
    
    def _normalize_attribution(self, attribution: np.ndarray) -> np.ndarray:
        """Normalize attribution values to [0, 1] range."""
        attr_min = attribution.min()
        attr_max = attribution.max()
        
        if attr_max > attr_min:
            return (attribution - attr_min) / (attr_max - attr_min)
        else:
            return np.zeros_like(attribution)
    
    def _create_attribution_overlay(
        self, 
        original_image: np.ndarray, 
        attribution: np.ndarray, 
        method: str,
        alpha: float = 0.6
    ) -> np.ndarray:
        """Create an overlay of attribution on original image."""
        import matplotlib.cm as cm
        
        # Get colormap for method
        cmap_name = self.colormaps.get(method, 'jet')
        cmap = cm.get_cmap(cmap_name)
        
        # Apply colormap to attribution
        heatmap = cmap(attribution)
        
        # Ensure original image is in the right format
        if len(original_image.shape) == 2:
            original_rgb = np.stack([original_image] * 3, axis=-1)
        elif len(original_image.shape) == 3 and original_image.shape[-1] == 1:
            original_rgb = np.repeat(original_image, 3, axis=-1)
        else:
            original_rgb = original_image
        
        # Normalize original image
        if original_rgb.max() > 1.0:
            original_rgb = original_rgb / 255.0
        
        # Create overlay
        overlay = alpha * heatmap[..., :3] + (1 - alpha) * original_rgb
        overlay = np.clip(overlay, 0, 1)
        
        return overlay
    
    def plot_attribution_statistics(
        self,
        attributions: Dict[str, np.ndarray],
        save_path: Optional[str] = None,
        figsize: Tuple[int, int] = (12, 8)
    ) -> plt.Figure:
        """Plot statistics about attribution methods."""
        fig, axes = plt.subplots(2, 2, figsize=figsize)
        
        # Attribution value distributions
        ax = axes[0, 0]
        for method, attr in attributions.items():
            attr_flat = attr.flatten()
            ax.hist(attr_flat, alpha=0.7, label=method, bins=50)
        ax.set_title('Attribution Value Distributions')
        ax.set_xlabel('Attribution Value')
        ax.set_ylabel('Frequency')
        ax.legend()
        
        # Attribution sparsity (percentage of high-value pixels)
        ax = axes[0, 1]
        sparsity_values = []
        methods = []
        for method, attr in attributions.items():
            threshold = np.percentile(attr.flatten(), 90)
            sparsity = np.mean(attr > threshold) * 100
            sparsity_values.append(sparsity)
            methods.append(method.replace('_', ' ').title())
        
        ax.bar(methods, sparsity_values)
        ax.set_title('Attribution Sparsity (Top 10% pixels)')
        ax.set_ylabel('Percentage (%)')
        ax.tick_params(axis='x', rotation=45)
        
        # Attribution intensity heatmap
        ax = axes[1, 0]
        if 'aggregated' in attributions:
            im = ax.imshow(attributions['aggregated'], cmap='RdYlBu_r')
            ax.set_title('Aggregated Attribution Heatmap')
            plt.colorbar(im, ax=ax)
        
        # Method correlations
        ax = axes[1, 1]
        if len(attributions) > 1:
            methods = list(attributions.keys())
            n_methods = len(methods)
            correlation_matrix = np.zeros((n_methods, n_methods))
            
            for i, method1 in enumerate(methods):
                for j, method2 in enumerate(methods):
                    if i == j:
                        correlation_matrix[i, j] = 1.0
                    else:
                        attr1 = attributions[method1].flatten()
                        attr2 = attributions[method2].flatten()
                        correlation = np.corrcoef(attr1, attr2)[0, 1]
                        correlation_matrix[i, j] = correlation if not np.isnan(correlation) else 0
            
            im = ax.imshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
            ax.set_xticks(range(n_methods))
            ax.set_yticks(range(n_methods))
            ax.set_xticklabels([m.replace('_', ' ').title() for m in methods], rotation=45)
            ax.set_yticklabels([m.replace('_', ' ').title() for m in methods])
            ax.set_title('Attribution Method Correlations')
            
            # Add correlation values to cells
            for i in range(n_methods):
                for j in range(n_methods):
                    ax.text(j, i, f'{correlation_matrix[i, j]:.2f}',
                           ha='center', va='center', color='white' if abs(correlation_matrix[i, j]) > 0.5 else 'black')
            
            plt.colorbar(im, ax=ax)
        
        plt.tight_layout()
        
        if save_path:
            fig.savefig(save_path, dpi=300, bbox_inches='tight')
        
        return fig

print("Attribution and visualization tools defined successfully!")


In [ ]:
# Configuration Management and Utilities
def create_default_config() -> Dict[str, Any]:
    """Create default configuration for CausalXray training."""
    config = {
        'experiment_name': 'causalxray_experiment',
        'output_dir': './experiments',
        'log_dir': './logs',
        'checkpoint_dir': './checkpoints',
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
        'random_seed': 42,
        
        # Model configuration
        'model': {
            'backbone': {
                'architecture': 'densenet121',
                'pretrained': True,
                'num_classes': 2,
                'feature_dims': [1024, 512, 256],
                'dropout_rate': 0.3
            }
        },
        
        # Causal configuration
        'causal': {
            'confounders': {
                'age': 1,
                'sex': 2,
                'view_position': 3
            },
            'hidden_dims': [256, 128],
            'dropout_rate': 0.3,
            'use_variational': False,
            'use_domain_adaptation': False
        },
        
        # Training configuration
        'training': {
            'batch_size': 32,
            'num_epochs': 100,
            'learning_rate': 1e-3,
            'weight_decay': 1e-4,
            'grad_clip': 1.0,
            'early_stopping': True,
            'patience': 20,
            'min_delta': 0.001
        },
        
        # Progressive training
        'progressive_training': {
            'enabled': True,
            'phase_epochs': [30, 30, 40]
        },
        
        # Data configuration
        'data': {
            'dataset': 'nih',
            'image_size': [224, 224],
            'normalize': True,
            'augmentation': True,
            'num_workers': 4,
            'pin_memory': True
        },
        
        # Loss configuration
        'loss': {
            'weights': {
                'classification': 1.0,
                'disentanglement': 0.3,
                'domain': 0.1,
                'attribution': 0.2
            },
            'use_focal': False,
            'focal_alpha': 1.0,
            'focal_gamma': 2.0
        },
        
        # Optimizer configuration
        'optimizer': {
            'type': 'adam',
            'lr': 1e-3,
            'weight_decay': 1e-4,
            'betas': [0.9, 0.999]
        },
        
        # Scheduler configuration
        'scheduler': {
            'enabled': True,
            'type': 'cosine',
            'T_max': 100,
            'eta_min': 1e-6
        },
        
        # Attribution configuration
        'attribution': {
            'patch_size': 16,
            'attribution_methods': ['intervention', 'counterfactual', 'gradcam']
        },
        
        # Logging configuration
        'logging': {
            'use_tensorboard': True,
            'use_wandb': False,
            'log_interval': 10
        }
    }
    
    return config


def save_config(config: Dict[str, Any], save_path: str):
    """Save configuration to YAML file."""
    import yaml
    
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    with open(save_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, indent=2)
    
    print(f"Configuration saved to {save_path}")


def load_config(config_path: str) -> Dict[str, Any]:
    """Load configuration from YAML file."""
    import yaml
    
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    print(f"Configuration loaded from {config_path}")
    return config


def merge_configs(base_config: Dict[str, Any], override_config: Dict[str, Any]) -> Dict[str, Any]:
    """Merge two configurations with override taking precedence."""
    import copy
    
    merged = copy.deepcopy(base_config)
    
    def recursive_update(d1, d2):
        for key, value in d2.items():
            if key in d1 and isinstance(d1[key], dict) and isinstance(value, dict):
                recursive_update(d1[key], value)
            else:
                d1[key] = value
    
    recursive_update(merged, override_config)
    return merged


def setup_logging_config(config: Dict[str, Any]) -> logging.Logger:
    """Setup comprehensive logging configuration."""
    import logging.config
    
    log_level = config.get('log_level', 'INFO')
    log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    
    logging_config = {
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'standard': {
                'format': log_format
            }
        },
        'handlers': {
            'console': {
                'level': log_level,
                'class': 'logging.StreamHandler',
                'formatter': 'standard',
                'stream': 'ext://sys.stdout'
            }
        },
        'loggers': {
            '': {  # root logger
                'level': log_level,
                'handlers': ['console'],
                'propagate': False
            }
        }
    }
    
    # Add file handler if log directory is specified
    if 'log_dir' in config:
        log_dir = Path(config['log_dir'])
        log_dir.mkdir(parents=True, exist_ok=True)
        
        logging_config['handlers']['file'] = {
            'level': log_level,
            'class': 'logging.FileHandler',
            'formatter': 'standard',
            'filename': str(log_dir / 'causalxray.log'),
            'mode': 'a'
        }
        logging_config['loggers']['']['handlers'].append('file')
    
    logging.config.dictConfig(logging_config)
    logger = logging.getLogger('CausalXray')
    
    return logger


# Utility Functions
def count_parameters(model: nn.Module) -> int:
    """Count the number of trainable parameters in a model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def get_model_size_mb(model: nn.Module) -> float:
    """Get model size in megabytes."""
    param_size = 0
    buffer_size = 0
    
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    
    size_mb = (param_size + buffer_size) / (1024 ** 2)
    return size_mb


def print_model_summary(model: nn.Module):
    """Print a summary of the model architecture."""
    total_params = count_parameters(model)
    model_size = get_model_size_mb(model)
    
    print(f"{'='*50}")
    print(f"Model Summary")
    print(f"{'='*50}")
    print(f"Architecture: {model.__class__.__name__}")
    print(f"Trainable Parameters: {total_params:,}")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"{'='*50}")


def save_training_history(history: Dict[str, List], save_path: str):
    """Save training history to JSON file."""
    import json
    
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    with open(save_path, 'w') as f:
        json.dump(history, f, indent=2)
    
    print(f"Training history saved to {save_path}")


def plot_training_history(
    history: Dict[str, List],
    save_path: Optional[str] = None,
    figsize: Tuple[int, int] = (12, 8)
) -> plt.Figure:
    """Plot training history."""
    fig, axes = plt.subplots(2, 2, figsize=figsize)
    
    # Loss curves
    ax = axes[0, 0]
    if 'train_total' in history and 'val_total' in history:
        ax.plot(history['train_total'], label='Train Loss')
        ax.plot(history['val_total'], label='Val Loss')
        ax.set_title('Loss Curves')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.legend()
        ax.grid(True)
    
    # Accuracy curves
    ax = axes[0, 1]
    if 'train_accuracy' in history and 'val_accuracy' in history:
        ax.plot(history['train_accuracy'], label='Train Accuracy')
        ax.plot(history['val_accuracy'], label='Val Accuracy')
        ax.set_title('Accuracy Curves')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Accuracy')
        ax.legend()
        ax.grid(True)
    
    # AUC curves (if available)
    ax = axes[1, 0]
    if 'train_auc' in history and 'val_auc' in history:
        ax.plot(history['train_auc'], label='Train AUC')
        ax.plot(history['val_auc'], label='Val AUC')
        ax.set_title('AUC Curves')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('AUC')
        ax.legend()
        ax.grid(True)
    
    # Learning rate (if available)
    ax = axes[1, 1]
    if 'learning_rate' in history:
        ax.plot(history['learning_rate'])
        ax.set_title('Learning Rate Schedule')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Learning Rate')
        ax.set_yscale('log')
        ax.grid(True)
    
    plt.tight_layout()
    
    if save_path:
        fig.savefig(save_path, dpi=300, bbox_inches='tight')
    
    return fig


def create_experiment_directory(experiment_name: str, base_dir: str = './experiments') -> Path:
    """Create experiment directory with timestamp."""
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    exp_dir = Path(base_dir) / f"{experiment_name}_{timestamp}"
    
    # Create subdirectories
    subdirs = ['checkpoints', 'logs', 'results', 'configs']
    for subdir in subdirs:
        (exp_dir / subdir).mkdir(parents=True, exist_ok=True)
    
    print(f"Created experiment directory: {exp_dir}")
    return exp_dir


def get_device_info() -> Dict[str, Any]:
    """Get information about available devices."""
    device_info = {
        'cuda_available': torch.cuda.is_available(),
        'device_count': 0,
        'current_device': None,
        'device_name': None,
        'memory_allocated': 0,
        'memory_reserved': 0
    }
    
    if torch.cuda.is_available():
        device_info['device_count'] = torch.cuda.device_count()
        device_info['current_device'] = torch.cuda.current_device()
        device_info['device_name'] = torch.cuda.get_device_name()
        device_info['memory_allocated'] = torch.cuda.memory_allocated() / (1024**3)  # GB
        device_info['memory_reserved'] = torch.cuda.memory_reserved() / (1024**3)  # GB
    
    return device_info


def print_device_info():
    """Print device information."""
    info = get_device_info()
    
    print(f"{'='*40}")
    print(f"Device Information")
    print(f"{'='*40}")
    print(f"CUDA Available: {info['cuda_available']}")
    
    if info['cuda_available']:
        print(f"Device Count: {info['device_count']}")
        print(f"Current Device: {info['current_device']}")
        print(f"Device Name: {info['device_name']}")
        print(f"Memory Allocated: {info['memory_allocated']:.2f} GB")
        print(f"Memory Reserved: {info['memory_reserved']:.2f} GB")
    else:
        print("Using CPU")
    
    print(f"{'='*40}")


# Data validation utilities
def validate_dataset_structure(data_dir: str, dataset_type: str = 'nih') -> bool:
    """Validate dataset directory structure."""
    data_path = Path(data_dir)
    
    if not data_path.exists():
        print(f"Error: Data directory does not exist: {data_dir}")
        return False
    
    if dataset_type == 'nih':
        required_files = ['images', 'Data_Entry_2017.csv']
        for file_name in required_files:
            if not (data_path / file_name).exists():
                print(f"Warning: {file_name} not found in {data_dir}")
    
    print(f"Dataset structure validation completed for {dataset_type}")
    return True


print("Configuration management and utilities defined successfully!")


In [ ]:
# Complete End-to-End CausalXray Pipeline

class CausalXrayPipeline:
    """Complete pipeline for CausalXray model training, evaluation, and inference."""
    
    def __init__(self, config: Optional[Dict[str, Any]] = None):
        """Initialize the CausalXray pipeline."""
        self.config = config or create_default_config()
        self.device = torch.device(self.config.get('device', 'cuda' if torch.cuda.is_available() else 'cpu'))
        
        # Set random seed
        set_seed(self.config.get('random_seed', 42))
        
        # Initialize components
        self.model = None
        self.trainer = None
        self.train_loader = None
        self.val_loader = None
        self.test_loader = None
        self.attribution_module = None
        self.visualizer = None
        
        # Setup logging
        self.logger = setup_logging_config(self.config)
        
        # Create experiment directory
        exp_name = self.config.get('experiment_name', 'causalxray_exp')
        self.exp_dir = create_experiment_directory(exp_name)
        self.config['output_dir'] = str(self.exp_dir)
    
    def setup_data(self, data_dir: str) -> Tuple[DataLoader, DataLoader, DataLoader]:
        """Setup data loaders for training, validation, and testing."""
        self.logger.info("Setting up data loaders...")
        
        # Validate dataset structure
        dataset_type = self.config['data'].get('dataset', 'nih')
        validate_dataset_structure(data_dir, dataset_type)
        
        # Create transforms
        train_transforms = CausalTransforms(
            mode='train',
            image_size=tuple(self.config['data']['image_size'])
        )
        val_transforms = CausalTransforms(
            mode='val',
            image_size=tuple(self.config['data']['image_size'])
        )
        test_transforms = CausalTransforms(
            mode='test',
            image_size=tuple(self.config['data']['image_size'])
        )
        
        # Create datasets
        if dataset_type == 'nih':
            dataset_class = NIHChestXray14
        elif dataset_type == 'rsna':
            dataset_class = RSNAPneumonia
        elif dataset_type == 'pediatric':
            dataset_class = PediatricDataset
        else:
            raise ValueError(f"Unsupported dataset type: {dataset_type}")
        
        # Confounder configuration
        confounder_config = self.config['causal'].get('confounders', {})
        
        train_dataset = dataset_class(
            data_dir=data_dir,
            split='train',
            transform=train_transforms,
            include_confounders=True,
            confounder_config={'categories': confounder_config}
        )
        
        val_dataset = dataset_class(
            data_dir=data_dir,
            split='val',
            transform=val_transforms,
            include_confounders=True,
            confounder_config={'categories': confounder_config}
        )
        
        test_dataset = dataset_class(
            data_dir=data_dir,
            split='test',
            transform=test_transforms,
            include_confounders=True,
            confounder_config={'categories': confounder_config}
        )
        
        # Create data loaders
        batch_size = self.config['training']['batch_size']
        num_workers = self.config['data'].get('num_workers', 4)
        
        self.train_loader = create_dataloader(
            train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
        )
        self.val_loader = create_dataloader(
            val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
        )
        self.test_loader = create_dataloader(
            test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
        )
        
        # Log dataset statistics
        self.logger.info(f"Train samples: {len(train_dataset)}")
        self.logger.info(f"Val samples: {len(val_dataset)}")
        self.logger.info(f"Test samples: {len(test_dataset)}")
        
        return self.train_loader, self.val_loader, self.test_loader
    
    def setup_model(self) -> nn.Module:
        """Setup the CausalXray model."""
        self.logger.info("Setting up CausalXray model...")
        
        # Create model configuration
        backbone_config = self.config['model']['backbone']
        causal_config = self.config['causal']
        
        # Create model
        self.model = CausalXrayModel(
            backbone_config=backbone_config,
            causal_config=causal_config
        )
        
        self.model = self.model.to(self.device)
        
        # Print model summary
        print_model_summary(self.model)
        
        return self.model
    
    def setup_training(self) -> CausalTrainer:
        """Setup the training framework."""
        if self.model is None:
            raise ValueError("Model must be set up before training")
        if self.train_loader is None or self.val_loader is None:
            raise ValueError("Data loaders must be set up before training")
        
        self.logger.info("Setting up training framework...")
        
        # Update config with experiment directory
        training_config = self.config.copy()
        training_config.update({
            'output_dir': str(self.exp_dir),
            'checkpoint_dir': str(self.exp_dir / 'checkpoints'),
            'log_dir': str(self.exp_dir / 'logs')
        })
        
        self.trainer = CausalTrainer(
            model=self.model,
            train_loader=self.train_loader,
            val_loader=self.val_loader,
            config=training_config,
            device=str(self.device),
            logger=self.logger
        )
        
        return self.trainer
    
    def train(self, num_epochs: Optional[int] = None, resume_from: Optional[str] = None) -> Dict[str, List]:
        """Train the CausalXray model."""
        if self.trainer is None:
            self.setup_training()
        
        num_epochs = num_epochs or self.config['training']['num_epochs']
        
        self.logger.info(f"Starting training for {num_epochs} epochs")
        
        # Save configuration
        config_path = self.exp_dir / 'configs' / 'training_config.yaml'
        save_config(self.config, str(config_path))
        
        # Print device info
        print_device_info()
        
        # Train model
        history = self.trainer.train(num_epochs=num_epochs, resume_from=resume_from)
        
        # Save training history
        history_path = self.exp_dir / 'results' / 'training_history.json'
        save_training_history(history, str(history_path))
        
        # Plot training history
        fig = plot_training_history(history)
        fig.savefig(self.exp_dir / 'results' / 'training_curves.png', dpi=300, bbox_inches='tight')
        plt.close(fig)
        
        self.logger.info("Training completed successfully!")
        
        return history
    
    def evaluate(self, checkpoint_path: Optional[str] = None) -> Dict[str, float]:
        """Evaluate the model on test dataset."""
        if self.model is None:
            self.setup_model()
        if self.test_loader is None:
            raise ValueError("Test data loader must be set up before evaluation")
        
        # Load checkpoint if provided
        if checkpoint_path:
            checkpoint = torch.load(checkpoint_path, map_location=self.device)
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.logger.info(f"Loaded checkpoint from {checkpoint_path}")
        elif self.trainer and hasattr(self.trainer, 'checkpoint_dir'):
            # Try to load best model from training
            best_model_path = self.trainer.checkpoint_dir / 'best_model.pth'
            if best_model_path.exists():
                checkpoint = torch.load(best_model_path, map_location=self.device)
                self.model.load_state_dict(checkpoint['model_state_dict'])
                self.logger.info(f"Loaded best model from training")
        
        self.logger.info("Evaluating model on test dataset...")
        
        self.model.eval()
        metrics_calculator = CausalMetrics()
        
        all_predictions = []
        all_labels = []
        
        with torch.no_grad():
            for batch in tqdm(self.test_loader, desc="Evaluating"):
                images = batch['image'].to(self.device)
                labels = batch['label'].to(self.device)
                
                outputs = self.model(images)
                predictions = outputs['probabilities']
                
                all_predictions.append(predictions.cpu().numpy())
                all_labels.append(labels.cpu().numpy())
        
        # Compute metrics
        all_predictions = np.vstack(all_predictions)
        all_labels = np.concatenate(all_labels)
        
        test_metrics = metrics_calculator.compute_epoch_metrics(
            all_predictions, all_labels, return_detailed=True
        )
        
        # Log results
        self.logger.info("Test Results:")
        for metric, value in test_metrics.items():
            self.logger.info(f"  {metric}: {value:.4f}")
        
        # Save results
        results_path = self.exp_dir / 'results' / 'test_results.json'
        with open(results_path, 'w') as f:
            json.dump(test_metrics, f, indent=2)
        
        return test_metrics
    
    def setup_attribution(self) -> CausalAttribution:
        """Setup causal attribution module."""
        if self.model is None:
            raise ValueError("Model must be set up before attribution")
        
        self.logger.info("Setting up causal attribution...")
        
        attribution_config = self.config.get('attribution', {})
        
        self.attribution_module = CausalAttribution(
            model=self.model,
            attribution_methods=attribution_config.get('attribution_methods', ['intervention', 'gradcam']),
            patch_size=attribution_config.get('patch_size', 16)
        )
        
        self.visualizer = AttributionVisualizer()
        
        return self.attribution_module
    
    def generate_attributions(
        self,
        image: Union[torch.Tensor, np.ndarray, str],
        target_class: Optional[int] = None,
        save_path: Optional[str] = None
    ) -> Dict[str, np.ndarray]:
        """Generate causal attributions for a single image."""
        if self.attribution_module is None:
            self.setup_attribution()
        
        self.logger.info("Generating causal attributions...")
        
        # Process input image
        if isinstance(image, str):
            # Load from file
            image_pil = Image.open(image)
            if image_pil.mode != 'RGB':
                image_pil = image_pil.convert('RGB')
            
            # Apply transforms
            transforms = CausalTransforms(mode='test', image_size=tuple(self.config['data']['image_size']))
            image_tensor = transforms(image_pil).unsqueeze(0).to(self.device)
            
            # Keep original for visualization
            original_image = np.array(image_pil)
            if len(original_image.shape) == 3 and original_image.shape[2] == 3:
                original_image = np.mean(original_image, axis=2)  # Convert to grayscale for visualization
            
        elif isinstance(image, np.ndarray):
            original_image = image.copy()
            # Convert to PIL and then tensor
            image_pil = Image.fromarray((image * 255).astype(np.uint8))
            transforms = CausalTransforms(mode='test', image_size=tuple(self.config['data']['image_size']))
            image_tensor = transforms(image_pil).unsqueeze(0).to(self.device)
            
        else:  # torch.Tensor
            image_tensor = image.to(self.device)
            if len(image_tensor.shape) == 3:
                image_tensor = image_tensor.unsqueeze(0)
            
            # Convert to numpy for visualization
            original_image = image_tensor.squeeze().cpu().numpy()
            if len(original_image.shape) == 3:
                original_image = np.mean(original_image, axis=0)
        
        # Generate attributions
        self.model.eval()
        with torch.no_grad():
            # Get model prediction
            model_output = self.model(image_tensor)
            predicted_class = torch.argmax(model_output['probabilities'], dim=1).item()
            confidence = torch.max(model_output['probabilities'], dim=1)[0].item()
            
            # Generate attributions
            attributions_tensor = self.attribution_module(image_tensor, target_class)
        
        # Convert to numpy
        attributions_np = {}
        for method, attr_tensor in attributions_tensor.items():
            if isinstance(attr_tensor, torch.Tensor):
                attributions_np[method] = attr_tensor.squeeze().cpu().numpy()
        
        # Create prediction info
        class_names = ['Normal', 'Pneumonia']
        prediction_info = {
            'predicted_class': class_names[predicted_class],
            'confidence': confidence,
            'probabilities': {
                'normal': model_output['probabilities'][0, 0].item(),
                'pneumonia': model_output['probabilities'][0, 1].item()
            }
        }
        
        # Visualize attributions
        if self.visualizer:
            fig = self.visualizer.visualize_attribution_comparison(
                original_image, attributions_np, prediction_info
            )
            
            if save_path:
                fig.savefig(save_path, dpi=300, bbox_inches='tight')
                self.logger.info(f"Attribution visualization saved to {save_path}")
            
            plt.show()
        
        return {
            'attributions': attributions_np,
            'prediction': prediction_info,
            'original_image': original_image
        }
    
    def inference(
        self,
        image_path: str,
        checkpoint_path: Optional[str] = None,
        show_attributions: bool = True
    ) -> Dict[str, Any]:
        """Perform inference on a single image with optional attribution visualization."""
        if self.model is None:
            self.setup_model()
        
        # Load checkpoint if provided
        if checkpoint_path:
            checkpoint = torch.load(checkpoint_path, map_location=self.device)
            self.model.load_state_dict(checkpoint['model_state_dict'])
        
        self.logger.info(f"Performing inference on {image_path}")
        
        # Load and preprocess image
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        transforms = CausalTransforms(mode='test', image_size=tuple(self.config['data']['image_size']))
        image_tensor = transforms(image).unsqueeze(0).to(self.device)
        
        # Inference
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(image_tensor)
            probabilities = outputs['probabilities'].cpu().numpy()[0]
            predicted_class = np.argmax(probabilities)
        
        # Get results
        class_names = ['Normal', 'Pneumonia']
        predicted_label = class_names[predicted_class]
        confidence = probabilities[predicted_class]
        
        results = {
            'predicted_class': predicted_label,
            'confidence': float(confidence),
            'probabilities': {
                'normal': float(probabilities[0]),
                'pneumonia': float(probabilities[1])
            },
            'image_path': image_path
        }
        
        print(f"\nInference Results:")
        print(f"Predicted Class: {predicted_label}")
        print(f"Confidence: {confidence:.4f}")
        print(f"Probabilities: Normal={probabilities[0]:.4f}, Pneumonia={probabilities[1]:.4f}")
        
        # Generate attributions if requested
        if show_attributions:
            attribution_results = self.generate_attributions(
                image_path,
                save_path=str(self.exp_dir / 'results' / 'attribution_visualization.png')
            )
            results['attributions'] = attribution_results['attributions']
        
        # Save results
        results_path = self.exp_dir / 'results' / 'inference_results.json'
        with open(results_path, 'w') as f:
            # Convert numpy arrays to lists for JSON serialization
            json_results = results.copy()
            if 'attributions' in json_results:
                json_results['attributions'] = {
                    k: v.tolist() if isinstance(v, np.ndarray) else v
                    for k, v in json_results['attributions'].items()
                }
            json.dump(json_results, f, indent=2)
        
        return results


# Convenience function for quick usage
def run_causalxray_experiment(
    data_dir: str,
    config: Optional[Dict[str, Any]] = None,
    num_epochs: Optional[int] = None,
    evaluate_model: bool = True,
    run_inference: bool = False,
    inference_image: Optional[str] = None
) -> Dict[str, Any]:
    """
    Run a complete CausalXray experiment.
    
    Args:
        data_dir: Path to dataset directory
        config: Configuration dictionary (uses default if None)
        num_epochs: Number of training epochs
        evaluate_model: Whether to evaluate on test set
        run_inference: Whether to run inference on sample image
        inference_image: Path to image for inference (uses sample if None)
        
    Returns:
        Dictionary containing experiment results
    """
    print("🚀 Starting CausalXray Experiment")
    print("=" * 50)
    
    # Initialize pipeline
    pipeline = CausalXrayPipeline(config)
    
    # Setup data
    pipeline.setup_data(data_dir)
    
    # Setup model
    pipeline.setup_model()
    
    # Train model
    training_history = pipeline.train(num_epochs=num_epochs)
    
    results = {
        'training_history': training_history,
        'experiment_dir': str(pipeline.exp_dir)
    }
    
    # Evaluate model
    if evaluate_model:
        test_metrics = pipeline.evaluate()
        results['test_metrics'] = test_metrics
    
    # Run inference
    if run_inference:
        if inference_image is None:
            # Create a sample image for demonstration
            sample_image = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
            sample_path = pipeline.exp_dir / 'sample_inference.png'
            Image.fromarray(sample_image).save(sample_path)
            inference_image = str(sample_path)
        
        inference_results = pipeline.inference(inference_image, show_attributions=True)
        results['inference_results'] = inference_results
    
    print("✅ CausalXray Experiment Completed Successfully!")
    print(f"📁 Results saved to: {pipeline.exp_dir}")
    
    return results

print("End-to-end pipeline defined successfully!")


In [ ]:
# Example 1: Quick experiment with default settings
def demo_quick_experiment():
    """Demonstrate quick CausalXray experiment."""
    print("Demo: Quick CausalXray Experiment")
    print("-" * 40)
    
    # Use sample data directory (will create sample data if not found)
    data_dir = "./sample_data"
    
    # Run with default configuration and reduced epochs for demo
    config = create_default_config()
    config['training']['num_epochs'] = 5  # Quick demo
    config['training']['batch_size'] = 8   # Small batch for demo
    
    # Run experiment
    results = run_causalxray_experiment(
        data_dir=data_dir,
        config=config,
        num_epochs=5,
        evaluate_model=True,
        run_inference=True
    )
    
    print(f"Experiment completed! Results saved to: {results['experiment_dir']}")
    return results


# Example 2: Step-by-step pipeline usage
def demo_step_by_step():
    """Demonstrate step-by-step pipeline usage."""
    print("Demo: Step-by-step CausalXray Pipeline")
    print("-" * 40)
    
    # Create custom configuration
    config = create_default_config()
    config['training']['num_epochs'] = 3
    config['training']['batch_size'] = 4
    
    # Initialize pipeline
    pipeline = CausalXrayPipeline(config)
    
    # Step 1: Setup data
    print("Step 1: Setting up data...")
    train_loader, val_loader, test_loader = pipeline.setup_data("./sample_data")
    
    # Step 2: Setup model
    print("Step 2: Setting up model...")
    model = pipeline.setup_model()
    
    # Step 3: Train model
    print("Step 3: Training model...")
    history = pipeline.train(num_epochs=3)
    
    # Step 4: Evaluate model
    print("Step 4: Evaluating model...")
    test_metrics = pipeline.evaluate()
    
    # Step 5: Generate attributions
    print("Step 5: Generating attributions...")
    # Create a sample image
    sample_image = np.random.randint(50, 200, (224, 224), dtype=np.uint8)
    sample_path = pipeline.exp_dir / 'demo_sample.png'
    Image.fromarray(sample_image).save(sample_path)
    
    # Generate attributions
    attribution_results = pipeline.generate_attributions(str(sample_path))
    
    print("Step-by-step demo completed!")
    return pipeline


# Example 3: Custom model configuration
def demo_custom_config():
    """Demonstrate custom configuration."""
    print("Demo: Custom Configuration")
    print("-" * 40)
    
    # Create custom configuration
    config = {
        'experiment_name': 'custom_causalxray_demo',
        'model': {
            'backbone': {
                'architecture': 'resnet50',  # Use ResNet instead of DenseNet
                'pretrained': True,
                'num_classes': 2,
                'feature_dims': [512, 256, 128],  # Smaller features
                'dropout_rate': 0.5
            }
        },
        'causal': {
            'confounders': {
                'age': 1,
                'sex': 2,
                'scanner_type': 3
            },
            'hidden_dims': [128, 64]
        },
        'training': {
            'batch_size': 16,
            'num_epochs': 5,
            'learning_rate': 5e-4,  # Lower learning rate
            'optimizer': {'type': 'adamw'},  # Use AdamW
            'scheduler': {'enabled': False}  # Disable scheduler
        },
        'data': {
            'dataset': 'rsna',  # Use RSNA dataset
            'image_size': [256, 256],  # Larger images
            'num_workers': 2
        },
        'attribution': {
            'patch_size': 32,  # Larger patches
            'attribution_methods': ['intervention', 'counterfactual']  # Only causal methods
        }
    }
    
    # Merge with defaults
    default_config = create_default_config()
    final_config = merge_configs(default_config, config)
    
    # Save configuration
    save_config(final_config, './custom_config.yaml')
    
    # Run experiment
    results = run_causalxray_experiment(
        data_dir="./sample_data",
        config=final_config,
        num_epochs=5
    )
    
    print("Custom configuration demo completed!")
    return results


# Example 4: Inference on multiple images
def demo_batch_inference(image_paths: List[str], checkpoint_path: str):
    """Demonstrate batch inference."""
    print("Demo: Batch Inference")
    print("-" * 40)
    
    # Initialize pipeline
    config = create_default_config()
    pipeline = CausalXrayPipeline(config)
    pipeline.setup_model()
    
    # Load checkpoint
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=pipeline.device)
        pipeline.model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Loaded model from {checkpoint_path}")
    else:
        print("Warning: Checkpoint not found, using random weights")
    
    # Run inference on multiple images
    all_results = []
    
    for i, image_path in enumerate(image_paths):
        if not os.path.exists(image_path):
            # Create sample image if path doesn't exist
            sample_image = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
            os.makedirs(os.path.dirname(image_path), exist_ok=True)
            Image.fromarray(sample_image).save(image_path)
            print(f"Created sample image: {image_path}")
        
        print(f"Processing image {i+1}/{len(image_paths)}: {image_path}")
        
        results = pipeline.inference(
            image_path,
            show_attributions=False  # Skip visualization for batch processing
        )
        
        all_results.append({
            'image_path': image_path,
            'prediction': results['predicted_class'],
            'confidence': results['confidence']
        })
    
    # Print summary
    print("\nBatch Inference Results:")
    print("=" * 50)
    for result in all_results:
        print(f"{result['image_path']}: {result['prediction']} ({result['confidence']:.3f})")
    
    return all_results


# Example 5: Model comparison
def demo_model_comparison():
    """Demonstrate comparison between different model configurations."""
    print("Demo: Model Comparison")
    print("-" * 40)
    
    # Configuration for DenseNet model
    densenet_config = create_default_config()
    densenet_config['experiment_name'] = 'densenet_comparison'
    densenet_config['training']['num_epochs'] = 3
    densenet_config['model']['backbone']['architecture'] = 'densenet121'
    
    # Configuration for ResNet model
    resnet_config = create_default_config()
    resnet_config['experiment_name'] = 'resnet_comparison'
    resnet_config['training']['num_epochs'] = 3
    resnet_config['model']['backbone']['architecture'] = 'resnet50'
    
    models_to_compare = [
        ('DenseNet-121', densenet_config),
        ('ResNet-50', resnet_config)
    ]
    
    comparison_results = {}
    
    for model_name, config in models_to_compare:
        print(f"\nTraining {model_name}...")
        
        # Run experiment
        results = run_causalxray_experiment(
            data_dir="./sample_data",
            config=config,
            num_epochs=3,
            evaluate_model=True
        )
        
        # Extract key metrics
        if 'test_metrics' in results:
            comparison_results[model_name] = {
                'accuracy': results['test_metrics'].get('accuracy', 0),
                'auc': results['test_metrics'].get('auc', 0),
                'f1': results['test_metrics'].get('f1', 0),
                'experiment_dir': results['experiment_dir']
            }
    
    # Print comparison
    print("\nModel Comparison Results:")
    print("=" * 60)
    print(f"{'Model':<15} {'Accuracy':<10} {'AUC':<10} {'F1-Score':<10}")
    print("-" * 60)
    
    for model_name, metrics in comparison_results.items():
        print(f"{model_name:<15} {metrics['accuracy']:<10.4f} {metrics['auc']:<10.4f} {metrics['f1']:<10.4f}")
    
    return comparison_results


# Utility function to run all demos
def run_all_demos():
    """Run all demonstration examples."""
    print("🎯 Running All CausalXray Demos")
    print("=" * 50)
    
    demos = [
        ("Quick Experiment", demo_quick_experiment),
        ("Step-by-step Pipeline", demo_step_by_step),
        ("Custom Configuration", demo_custom_config)
    ]
    
    demo_results = {}
    
    for demo_name, demo_func in demos:
        try:
            print(f"\n🏃 Running: {demo_name}")
            result = demo_func()
            demo_results[demo_name] = "✅ Completed Successfully"
            print(f"✅ {demo_name} completed successfully!")
            
        except Exception as e:
            demo_results[demo_name] = f"❌ Failed: {str(e)}"
            print(f"❌ {demo_name} failed: {str(e)}")
    
    # Print summary
    print("\n" + "=" * 50)
    print("Demo Summary:")
    print("=" * 50)
    for demo_name, status in demo_results.items():
        print(f"{demo_name}: {status}")
    
    return demo_results

# Print usage instructions
print("""
🎉 CausalXray Framework Setup Complete!

The notebook now contains everything needed to:
✅ Install dependencies and setup environment
✅ Load and preprocess chest X-ray datasets  
✅ Train CausalXray models with progressive training
✅ Evaluate model performance with comprehensive metrics
✅ Generate causal attributions and visualizations
✅ Run end-to-end experiments with full pipeline

Quick Start Examples:
1. run_all_demos() - Run all demonstration examples
2. demo_quick_experiment() - Quick 5-epoch training demo
3. demo_step_by_step() - Detailed step-by-step walkthrough
4. demo_custom_config() - Custom model configuration example

For a complete experiment, run:
results = run_causalxray_experiment(
    data_dir="path/to/your/data",
    num_epochs=100,
    evaluate_model=True,
    run_inference=True
)
""")
